In [2]:
!pip install transformers
!pip install fugashi
!pip install unidic-lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 kB 10.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658834 sha256=65fe67bb9de7f98075ef70d89a35398fbe3c6c8ea79e3e677947983019759015
  Stored in directory: /root/.cache/pip/wheels/56/9c/4f/2c115e896b4b6c584039ca19de3581d333856782ef

In [3]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# ファイルを/content配下にコピー
!cp "/content/drive/MyDrive/train-v1.1.json" "/content/"
!cp "/content/drive/MyDrive/valid-v1.1.json" "/content/"

# データ読み込み
import json

def load_jsts_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f.readlines()]
    return data

train_data_path = "/content/train-v1.1.json"
valid_data_path = "/content/valid-v1.1.json"

train_data = load_jsts_data(train_data_path)
valid_data = load_jsts_data(valid_data_path)

# 課題1
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, spearmanr

def get_sentence_embedding(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    return torch.mean(embeddings, dim=1).squeeze()

tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-v2")
valid_data = load_jsts_data(valid_data_path)

true_similarities = []
predicted_similarities = []

for item in valid_data:
    sent1 = item["sentence1"]
    sent2 = item["sentence2"]
    label = float(item["label"])

    embedding1 = get_sentence_embedding(sent1, tokenizer, model)
    embedding2 = get_sentence_embedding(sent2, tokenizer, model)

    cosine_similarity = 1 - cosine(embedding1.detach().numpy(), embedding2.detach().numpy())
    scaled_similarity = cosine_similarity * 5

    true_similarities.append(label)
    predicted_similarities.append(scaled_similarity)

# Pearson相関係数とSpearman相関係数を計算
pearson_corr = pearsonr(true_similarities, predicted_similarities)
spearman_corr = spearmanr(true_similarities, predicted_similarities)

print("Pearson correlation:", pearson_corr)
print("Spearman correlation:", spearman_corr)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Pearson correlation: PearsonRResult(statistic=0.6394918528543447, pvalue=2.3489912319898795e-168)
Spearman correlation: SignificanceResult(statistic=0.6321073119018878, pvalue=2.2699922828499695e-163)


In [4]:
# 評価指標を指定されたフォーマットで出力
with open("evaluation_scores.jsonl", "w") as f:
    f.write(json.dumps({"metrics": "Pearson correlation", "score": pearson_corr}) + "\n")
    f.write(json.dumps({"metrics": "Spearman correlation", "score": spearman_corr}) + "\n")